In [1]:
import torch
import torch.nn as nn
import copy

import torchmetrics

from conplex_dti.model.architectures import SimpleCoembeddingNoSigmoid

from conplex_dti.dataset.datamodules import DTIDataModule
from conplex_dti.featurizer import get_featurizer


In [2]:
# Necessary for TransferCoembedding class
# TODO: eventually move TransferCoembedding class to architectures.py
class Cosine(nn.Module):
    def forward(self, x1, x2):
        return nn.CosineSimilarity()(x1, x2)

DISTANCE_METRICS = {"Cosine": Cosine}
ACTIVATIONS = {"ReLU": nn.ReLU }


# Create a new model that has 2 layers of tensors instead of 1.
class TransferCoembedding(nn.Module):
    def __init__(
        self,
        pre_trained_model:SimpleCoembeddingNoSigmoid,
        drug_shape=2048,
        target_shape=1024,
        latent_dimension=1024,
        latent_activation="ReLU",
        latent_distance="Cosine",
        classify=True,
    ):
        super().__init__()
        # TODO: initialize these all baased on the pre-trained model
        self.drug_shape = drug_shape
        self.target_shape = target_shape
        self.latent_dimension = latent_dimension
        self.do_classify = classify
        self.latent_activation = ACTIVATIONS[latent_activation]

        self.drug_projector = nn.Sequential(
            nn.Linear(self.drug_shape, latent_dimension),     # [0]
            self.latent_activation(),                         # [1]
            # ADD AN ADDITIONAL LAYER AND ACTIVATION FUNCTION
            nn.Linear(latent_dimension, latent_dimension),    # [2]
            self.latent_activation()                          # [3]
        )

        # initialize layer 0 from pre-trained model:
        self.drug_projector[0] = copy.deepcopy(pre_trained_model.drug_projector[0])
        # initialize layer 2 randomly
        nn.init.xavier_normal_(self.drug_projector[2].weight)

        self.target_projector = nn.Sequential(
            nn.Linear(self.target_shape, latent_dimension),
            self.latent_activation(),
            # ADD AN ADDITIONAL LAYER AND ACTIVATION FUNCTION
            nn.Linear(latent_dimension, latent_dimension),
            self.latent_activation()
        )

        # initialize layer 0 from pre-trained model:
        self.target_projector[0] = copy.deepcopy(pre_trained_model.target_projector[0])
        # initialize layer 2 randomly
        nn.init.xavier_normal_(self.target_projector[2].weight)

        # freeze the first layers (neural net ) of the target and drug projectors
        for idx, param in enumerate(self.parameters()):
            if idx == 0 or idx == 1 or idx == 4 or idx == 5:
                param.requires_grad = False


        if self.do_classify: # if True:
            self.distance_metric = latent_distance # "Cosine"
            self.activator = DISTANCE_METRICS[self.distance_metric]() # gives it the Cosine activator function that was written

    def forward(self, drug, target):
        if self.do_classify: # if True:
            return self.classify(drug, target)
        else:
            return self.regress(drug, target)

    def classify(self, drug, target):
        drug_projection = self.drug_projector(drug)
        target_projection = self.target_projector(target)

        distance = self.activator(drug_projection, target_projection)
        return distance.squeeze()

    def regress(self, drug, target):
        drug_projection = self.drug_projector(drug)
        target_projection = self.target_projector(target)

        inner_prod = torch.bmm(
            drug_projection.view(-1, 1, self.latent_dimension),
            target_projection.view(-1, self.latent_dimension, 1),
        ).squeeze()
        return inner_prod.squeeze()


In [3]:
# TODO: want to try learning / training on three different models

# 1. TransferCoembedding model with two unfrozen layers
# 2. original model with only RELU layer unfrozen
# 3. original model with lower learning rate


# TODO: eventually do do weights and biases logging
# TODO: eventually do logg instead of print statements
# TODO: eventually add timing

In [4]:
DEVICE = torch.device("cpu")

In [5]:
# recreate lines 309 -> ~322 from train.py

# load the featurizers 
drug_featurizer = get_featurizer("MorganFeaturizer")
# load from pre-trained prot-bert model
target_featurizer = get_featurizer("ProtBertFeaturizer")


# load data into a drug-target interaction datamodule (combo of dataset/dataloader)
datamodule = DTIDataModule(data_dir = "./data/", drug_featurizer=drug_featurizer, target_featurizer=target_featurizer, batch_size=2)
datamodule.prepare_data()
datamodule.setup()

# from the datamodules, get dataloaders to use for training, testing, and validation
training_generator = datamodule.train_dataloader()
validation_generator = datamodule.val_dataloader()
testing_generator = datamodule.test_dataloader()

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Drug and target featurizers already exist
ProtBert: 100%|██████████| 2/2 [00:00<00:00, 4495.50it/s]


In [6]:
# load previous model from state dict!
pre_trained_model = SimpleCoembeddingNoSigmoid(2048, 1024, 1024)  # TODO: drug_featurizer.shape, target_featurizer.shape 2048, 1024, 1024
pre_trained_model.load_state_dict(torch.load("../pre_trained_model/models/ConPLex_v1_BindingDB.pt", map_location="cpu"))

<All keys matched successfully>

In [7]:
# TODO: move from notebook to architectures file 
# from conplex_dti.model.architectures import TransferCoembedding 
model = TransferCoembedding(pre_trained_model)
model = model.to(DEVICE)

In [8]:
# optimizer:
LEARNING_RATE = 0.01 # not same as sam, larger
RESET_AFTER_EPOCHS = 10 # aka lr_t0 from config file TODO: play with this, currently same as sam
opt = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE) # AdamW is Adam with weight decay
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(opt, T_0=RESET_AFTER_EPOCHS)



# More variables
NUM_EPOCHS = 3 # TODO: play with this
VALIDATE_AFTER_EPOCHS = 1 # TODO: play with this
CLASSIFY = True # 309
WATCH_METRIC = "val/aupr" # 310


SAVE_DIRECTORY = "./saved_models/" # TODO: eventually change this
RUN_ID = "test_run" # TODO: eventually change this
from pathlib import Path

In [9]:
# keep track of best model:
max_metric = 0
model_max = copy.deepcopy(model) # currently last (2) layers are random


In [10]:
# line 413 -> 423, NOT doing contrastive learning

# Using Binary Cross Entropy loss function because we are trying to predict a binary label (binding or no binding):
loss_fct = torch.nn.BCELoss() 

# Keep track of how well the model is doing (on each epoch)
val_metrics = {"val/aupr": torchmetrics.AveragePrecision, "val/auroc": torchmetrics.AUROC}
test_metrics = {"test/aupr": torchmetrics.AveragePrecision, "test/auroc": torchmetrics.AUROC}


In [11]:
# FUNCTIONS FOR TRAINING:
import numpy as np
from torch.autograd import Variable

def step(model, batch, device=None):
    if device is None:
        device = torch.device("cpu")

    drug, target, label = batch  # target is (D + N_pool)
    pred = model(drug.to(device), target.to(device))
    label = Variable(torch.from_numpy(np.array(label)).float()).to(device)
    return pred, label



In [12]:
# FUNCTIONS FOR TESTING:
def test(model, data_generator, metrics, device=None, classify=True):
    if device is None:
        device = torch.device("cpu")

    metric_dict = {}

    for k, met_class in metrics.items():
        if classify:
            met_instance = met_class(task="binary")
        else:
            met_instance = met_class()
        met_instance.to(device)
        met_instance.reset()
        metric_dict[k] = met_instance

    model.eval()

    for i, batch in enumerate(data_generator):
        pred, label = step(model, batch, device)
        if classify:
            label = label.int()
        else:
            label = label.float()

        for _, met_instance in metric_dict.items():
            met_instance(pred, label)

    results = {}
    for k, met_instance in metric_dict.items():
        res = met_instance.compute()
        results[k] = res

    for met_instance in metric_dict.values():
        met_instance.to("cpu")

    return results


In [14]:

print(f"Beginning Training")

torch.backends.cudnn.benchmark = True

for epo in range(NUM_EPOCHS): # TODO: eventually do tqdm (progress bar)

    print(f"Trainign epoch is {epo + 1}")

    # Training
    model.train()
    for i, batch in enumerate(training_generator):
        print(f"batch number {i + 1}")
        pred, label = step(model, batch, DEVICE)  # batch is (2048, 1024, 1)
        loss = loss_fct(pred, label)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
        # break
    lr_scheduler.step()
    print(f"loss was {loss.cpu().detach().numpy():8f}. Updating learning rate to {lr_scheduler.get_lr()[0]:8f}")


    # Validation
    if epo % VALIDATE_AFTER_EPOCHS == 0:
        with torch.set_grad_enabled(False):
            val_results = test(
                model = model,
                data_generator = validation_generator,
                metrics = val_metrics,
                device = DEVICE,
                classify = CLASSIFY,
            )
            val_results["epoch"] = epo 
            
            if val_results[WATCH_METRIC] > max_metric:
                print(f"Validation AUPR {val_results[WATCH_METRIC]:8f} > previous max {max_metric:8f}")
                model_max = copy.deepcopy(model)
                max_metric = val_results[WATCH_METRIC]
                model_save_path = Path(
                    f"{SAVE_DIRECTORY}/{RUN_ID}_best_model_epoch{epo:02}.pt"
                )
                torch.save(
                    model_max.state_dict(),
                    model_save_path,
                )
                print(f"Saving checkpoint model to {model_save_path}")
                
            print(f"Validation at Epoch {epo + 1}")
            for k, v in val_results.items():
                if not k.startswith("_") and not k.startswith("epoch"):
                    print(f"{k}: {v}")


Beginning Training
Trainign epoch is 1
batch number 1
batch number 2
batch number 3
batch number 4
loss was 1.577311. Updating learning rate to 0.009755
Validation AUPR 0.792857 > previous max 0.000000
Saving checkpoint model to saved_models/test_run_best_model_epoch00.pt
Validation at Epoch 1
val/aupr: 0.7928571701049805
val/auroc: 0.6875
Trainign epoch is 2
batch number 1
batch number 2
batch number 3
batch number 4
loss was 0.715084. Updating learning rate to 0.009045
Validation AUPR 0.804167 > previous max 0.792857
Saving checkpoint model to saved_models/test_run_best_model_epoch01.pt
Validation at Epoch 2
val/aupr: 0.8041666746139526
val/auroc: 0.8125
Trainign epoch is 3
batch number 1
batch number 2
batch number 3
batch number 4
loss was 0.501459. Updating learning rate to 0.007939
Validation AUPR 1.000000 > previous max 0.804167
Saving checkpoint model to saved_models/test_run_best_model_epoch02.pt
Validation at Epoch 3
val/aupr: 1.0
val/auroc: 1.0


In [15]:
# Testing
print("Beginning testing")
try:
    with torch.set_grad_enabled(False):
        model_max = model_max.eval()

        test_results = test(
            model = model_max,
            data_generator = testing_generator,
            metrics = test_metrics,
            device = DEVICE,
            classify = CLASSIFY,
        )
        
        test_results["epoch"] = epo + 1

        print("Final Testing")
        for k, v in test_results.items():
            if not k.startswith("_") and not k.startswith("epoch"):
                print(f"{k}: {v}")

        model_save_path = Path(f"{SAVE_DIRECTORY}/{RUN_ID}_best_model.pt")
        torch.save(
            model_max.state_dict(),
            model_save_path,
        )
        print(f"Saving final model to {model_save_path}")

except Exception as e:
    print(f"Testing failed with exception {e}")

Beginning testing
Final Testing
test/aupr: 1.0
test/auroc: 1.0
Saving final model to saved_models/test_run_best_model.pt
